Часть by Crowdcourcing engineer (Safin Dim).

Дополнял @mightynehgbor (оставлял свои комментарии)

In [ ]:
#!pip install toloka-kit
#!pip install crowd-kit
#!pip install httpx==0.27.2 --force-reinstall

  Using cached httpx-0.27.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached anyio-4.7.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached certifi-2024.12.14-py3-none-any.whl.metadata (2.3 kB)
  Using cached httpcore-1.0.7-py3-none-any.whl.metadata (21 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached exceptiongroup-1.2.2-py3-none-any.whl.metadata (6.6 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
Using cached httpx-0.27.2-py3-none-any.whl (76 kB)
Using cached httpcore-1.0.7-py3-none-any.whl (78 kB)
Using cached anyio-4.7.0-py3-none-any.whl (93 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
Using cached certifi-2024.12.14-py3-none-any.whl (164 kB)
Using cached exceptiongroup-1.2.2-py3-none-any.whl (16 kB)
Using cached h11-0.14.0-py3-none-any.whl 

In [ ]:
import toloka.client as toloka
import toloka.client.project.template_builder as tb
from crowdkit.aggregation import MajorityVote, DawidSkene
import datetime
import numpy as np
import pandas as pd
import os

import typing as tp

# Инициализируем константы

In [ ]:
from google.colab import userdata

tasks_token = userdata.get('TOKEN')  #os.environ["TOLOKA_TOKEN"]


In [ ]:
toloka_client = toloka.TolokaClient(tasks_token, url = 'https://tasks.yandex.ru')
print(toloka_client.get_requester())  #  Убедиться, что все рубли на месте

Requester(_unexpected={}, id='1490a12c879f85f22f985bc0846df6e3', balance=Decimal('700.0000'), public_name={'EN': 'ШАД-187'}, company=Requester.Company(_unexpected={}, id='151', superintendent_id='64ef54a8b2a44461537f06b121533c42'))


In [ ]:
INPUT_URL_COL = "downloadUrl"
LABEL_COL = "is_conifer"
HINT_COL = "hint"
RESULT_COL = "result"


POSITIVE_LABEL = True
NEGATIVE_LABEL = False

В этом задании наша команда должны создать проект по разметке изображений. Для начала, инициализируем класс проекта названием и описанием, которые будут видны исполнителям:

In [ ]:
new_project = toloka.Project(
    public_name='Хвойное или лиственное дерево?',
    public_description='Посмотрите на изображение и определите, является ли представленное дерево хвойным.',
)

Зададим спецификацию входных данных:

In [ ]:
input_specification = {
    INPUT_URL_COL: toloka.project.UrlSpec(required=True)
}

Комментарий mightyneighbor:
- лучше создать отдельное имя для результирующей колонки, потому что в ханипотах у нас будет похожее название для истинного ответа

In [ ]:
output_specification = {
    RESULT_COL: toloka.project.BooleanSpec(
        required=True,
        allowed_values = [POSITIVE_LABEL, NEGATIVE_LABEL])
}

In [ ]:
image_viewer = tb.ImageViewV1(tb.InputData(INPUT_URL_COL), max_width=140, ratio=[1, 1], rotatable=True)

radio_group_field = tb.ButtonRadioGroupFieldV1(
    tb.OutputData(RESULT_COL),
    [
        tb.GroupFieldOption(POSITIVE_LABEL, 'Является хвойным', hint="Хвойные деревья обладают хвоей, имеют конусообразную форму кроны, один ствол, формируются шишки"),  # TODO придумать четкие емкие подсказки что такое хвойное дерево на снимке чтобы толокер смог обратиться
        tb.GroupFieldOption(NEGATIVE_LABEL, 'Не является хвойным', hint="Деревья, не являющиеся хвойными, обладают плоскоми листьями, а не хвоей. Их крона, как правило, более пышная"), # TODO
    ],
    validation=tb.RequiredConditionV1(hint='Выберите один из вариантов'),
)

task_width_plugin = tb.TolokaPluginV1(
    kind='scroll',
    task_width=250,
)

hot_keys_plugin = tb.HotkeysPluginV1(
    key_1=tb.SetActionV1(tb.OutputData(RESULT_COL), POSITIVE_LABEL),
    key_2=tb.SetActionV1(tb.OutputData(RESULT_COL), NEGATIVE_LABEL),
)

project_interface = toloka.project.TemplateBuilderViewSpec(
    view=tb.ListViewV1([image_viewer, radio_group_field]),
    plugins=[task_width_plugin, hot_keys_plugin],
)

In [ ]:
new_project.task_spec = toloka.project.task_spec.TaskSpec(
    input_spec=input_specification,
    output_spec=output_specification,
    view_spec=project_interface,
)

In [ ]:
from IPython.core.display import display, HTML

with open('tree_instruction_.html','r') as file:
    public_instructions = file.read()
display(HTML(public_instructions))

In [ ]:
new_project.public_instructions = public_instructions

In [ ]:
new_project = toloka_client.create_project(new_project)

Итак, более-менее разумное описание сделано, проект создан. Переходим к более содержательной части -- прописыванию пулов и их правил. Благо, у нас есть размеченные данные, которые мы будем активно использовать для обучения, экзамена и ханипотов.

Для начала проведем необходимый анализ. Для разметки боевых данных нам необходимо организовать контроль качества, и хочется оставить в нём ханипоты как один из методов. При этом для здорового проекта необходимо, чтобы их количество было достаточно большим (иначе возможна ситуация, когда ответы на них будут заучены толокерами и качество упадёт). Поэтому постараемся 100+ данных из числа размеченных выделить на ханипоты. Оставшиеся

In [ ]:
training_data = pd.read_csv('/content/IRIS_training.tsv', sep='\t', names=[LABEL_COL, INPUT_URL_COL, HINT_COL])
training_data


,is_conifer,downloadUrl,hint
0,False,https://new-projects-team-public.s3.yandex.net...,"пучковатая форма, нет конуса - дерево не хвойное"
1,True,https://new-projects-team-public.s3.yandex.net...,"широкие, разветвленные структуры ветвей - хвой..."
2,True,https://new-projects-team-public.s3.yandex.net...,это хвойное дерево - в тени видна характерная ...
3,True,https://new-projects-team-public.s3.yandex.net...,здесь хвойное дерево - они чаще образуют окруж...
4,False,https://new-projects-team-public.s3.yandex.net...,на этом снимке характерные для лиственного дер...
5,False,https://new-projects-team-public.s3.yandex.net...,дерево лиственное - отчетливо видно плоские ли...
6,True,https://new-projects-team-public.s3.yandex.net...,хвойное дерево - силует окружности и форма конуса
7,False,https://new-projects-team-public.s3.yandex.net...,лиственное дерево - видим объемные ветви с пло...
8,True,https://new-projects-team-public.s3.yandex.net...,хвойное дерево - пирамидальная форма кроны
9,False,https://new-projects-team-public.s3.yandex.net...,лиственное дерево - крона сверху обычно имеет ...


In [ ]:
training = toloka.Training(
    project_id=new_project.id,
    private_name='Обучение классификации хвойных деревьев',
    may_contain_adult_content=False,
    assignment_max_duration_seconds=10*30,
    mix_tasks_in_creation_order=False,
    shuffle_tasks_in_task_suite=True,
    training_tasks_in_task_suite_count=30,
    task_suites_required_to_pass=1, # это количество страниц, нам нужно не больше чем количество всех заданий поделенное на количество заданий на странице
    retry_training_after_days=5,
    inherited_instructions=True,
)

In [ ]:
training = toloka_client.create_training(training)

training_tasks = [
    toloka.Task(
        input_values={INPUT_URL_COL: row[INPUT_URL_COL]},
        pool_id=training.id,
        known_solutions=[toloka.task.BaseTask.KnownSolution(
            output_values={RESULT_COL: row[LABEL_COL]})],
        message_on_unknown_solution=row[HINT_COL],
        )
    for index, row in training_data.iterrows()
]

In [ ]:
train_result = toloka_client.create_tasks(training_tasks, allow_defaults=True)

100%|██████████| 100/100 [00:03<00:00, 27.56it/s]


In [ ]:
exam_pool = toloka.Pool(
    project_id=new_project.id,
    private_name='Экзаменационный пул от ' + datetime.datetime.now().strftime('%Y-%m-%d'),
    may_contain_adult_content=False,
    reward_per_assignment=1,
    assignment_max_duration_seconds=600,
    will_expire=datetime.datetime.utcnow() + datetime.timedelta(days=365),
    defaults=toloka.pool.Pool.Defaults(default_overlap_for_new_task_suites=50),
    private_comment='Настраиваем экзамен для классификации хвойных деревьев',
)

exam_pool.set_mixer_config(
    golden_tasks_count=20,
)

exam_pool.filter = (
    (
        (toloka.filter.ClientType == 'BROWSER')  # не обязательно кмк, потому что с телефона такие маленькие картинки как будто даже удобнее разглядывать
    ) &
    (toloka.filter.Languages.in_('RU')) &
    (
        toloka.filter.RegionByPhone.in_(225)
    )
)

exam_pool.set_training_requirement(training_pool_id=training.id, training_passing_skill_value=93)

In [ ]:
EXAM_SKILL = next(toloka_client.get_skills("passed_exam"), None)

if EXAM_SKILL is None:
    EXAM_SKILL = toloka_client.create_skill(
        name='passed_exam',
        hidden=True,
)
EXAM_SKILL

Skill(_unexpected={'deprecated': False, 'global': False}, name='passed_exam', private_comment=None, hidden=True, skill_ttl_hours=None, training=None, public_name={'EN': 'passed_exam'}, public_requester_description=None, owner=Owner(_unexpected={}, id='1490a12c879f85f22f985bc0846df6e3', myself=True, company_id='151'), id='10791', created=datetime.datetime(2024, 12, 20, 19, 44, 58, 277000, tzinfo=datetime.timezone.utc))

In [ ]:
exam_pool.quality_control.add_action(
    collector=toloka.collectors.GoldenSet(history_size=15),
    conditions=[
        toloka.conditions.GoldenSetCorrectAnswersRate > 70,
        toloka.conditions.GoldenSetAnswersCount >= 15
    ],
    action=toloka.actions.SetSkill(skill_id=EXAM_SKILL.id, skill_value=1)
)

In [ ]:
exam_pool = toloka_client.create_pool(exam_pool)

In [ ]:
exam_data = pd.read_csv('./IRIS_exam.tsv', sep='\t', names=[LABEL_COL, INPUT_URL_COL])

In [ ]:
exam_tasks = [
    toloka.Task(
        pool_id=exam_pool.id,
        input_values={INPUT_URL_COL: row[INPUT_URL_COL]},
        known_solutions = [
            toloka.task.BaseTask.KnownSolution(
                output_values={RESULT_COL: row[LABEL_COL]}
            )
        ]
    )
    for index, row in exam_data.iterrows()
]

In [ ]:
prepared_exam_tasks = toloka_client.create_tasks(exam_tasks, allow_defaults=True)

100%|██████████| 100/100 [00:03<00:00, 28.79it/s]


Далее надо правильным образом обработать main пул, где будет уже боевая разметка. При этом у нас остаётся максимум 160 размеченных картинок для ханипотов (4 ушло на инструкцию, 15 на обучение, 21 на экзамен). Если давать один ханипот на 20 заданий, то нам придётся создать около 20 пулов, чтобы наборы толокеров не пересекались и получилось избежать фрода. Правда, тогда вообще не понятно, сколько платить толокерам за страницу... С учётом необходимости ещё и вводить перекрытие (хотелось бы динамическое), нам потребуется примерно в 3 раза больше денег + комиссия. То есть мы, выдавая 100 заданий на страницу, и оплачивая это 1 рублём, будем вынуждены потратить больше, чем есть в нашем бюджете (990 рублей без учёта налогов, оплаты экзамена и ханипотов). Потребуется опыт жёсткого скама работяг, нужно всё это обсудить.

Создадим много пулов для разметки, чтобы использовать на них один набор ханипотов, и не допускать исполнителей размечать данные из разных пулов. Тогда нам потребуется при выполнении задания из основного пула сразу давать толокеру навык, с которым он НЕ будет допускаться до других пулов. Я не знаю, как сделать это не костыльно...

Внизу пытался автоматизировать разделение на пулы, но ловлю статус 500 (ошибка на серваке))) ), так что перестал пытаться и думаю просто разделим сами на меньшее количество пулов, для дебага возьмем пул с 1000 картинками, остальное разделим на 2-5к картинок, и пускай юзеры повторяются, наверное можно просто в конце их приостанавливать просто в пулах с помощью временной блокировки а не присваивания навыка, потому что такое количество навыков у меня не получилось запихать в пул. Если создавать пул без навыков - все работает, а с навыками - ловлю ошибки (см внизу что за ошибки)

In [ ]:
main_pool = toloka.Pool(
    project_id=new_project.id,
    private_name='Боевой пул от ' + datetime.datetime.now().strftime('%Y-%m-%d'),
    may_contain_adult_content=True,
    reward_per_assignment=1,
    assignment_max_duration_seconds=500,
    will_expire=datetime.datetime.utcnow() + datetime.timedelta(days=365),
    defaults=toloka.pool.Pool.Defaults(default_overlap_for_new_task_suites=3),
    private_comment='Боевая разметка спорных картинок',
    filter=toloka.filter.FilterAnd([
        toloka.filter.Skill('passed_exam') >= 1  # Фильтр по навыку
    ]),
)
main_pool.set_mixer_config(
    real_tasks_count=30
)
main_pool.filter = (
    (
        (toloka.filter.ClientType == 'BROWSER')
    ) &
    (toloka.filter.Languages.in_('RU')) &
    (
        toloka.filter.RegionByPhone.in_(225)
    )
)
main_pool.set_training_requirement(training_pool_id=training.id, training_passing_skill_value=90)

In [ ]:
main_data = pd.read_csv('/content/images_with_contradictions.tsv', sep='\t', names=[INPUT_URL_COL])
# main_data = main_data.dropna()
main_data

,downloadUrl
0,https://new-projects-team-public.s3.yandex.net...
1,https://new-projects-team-public.s3.yandex.net...
2,https://new-projects-team-public.s3.yandex.net...
3,https://new-projects-team-public.s3.yandex.net...
4,https://new-projects-team-public.s3.yandex.net...
...,...
9226,https://new-projects-team-public.s3.yandex.net...
9227,https://new-projects-team-public.s3.yandex.net...
9228,https://new-projects-team-public.s3.yandex.net...
9229,https://new-projects-team-public.s3.yandex.net...


In [ ]:
honeypots = pd.read_csv('/content/hw_3_markup_data.tsv', sep='\t', names=[LABEL_COL, INPUT_URL_COL])
# honeypots = honeypots.dropna()
honeypots

,is_conifer,downloadUrl
0,False,https://new-projects-team-public.s3.yandex.net...
1,False,https://new-projects-team-public.s3.yandex.net...
2,True,https://new-projects-team-public.s3.yandex.net...
3,False,https://new-projects-team-public.s3.yandex.net...
4,False,https://new-projects-team-public.s3.yandex.net...
...,...,...
142,True,https://new-projects-team-public.s3.yandex.net...
143,False,https://new-projects-team-public.s3.yandex.net...
144,True,https://new-projects-team-public.s3.yandex.net...
145,True,https://new-projects-team-public.s3.yandex.net...


In [ ]:
main_pool.set_mixer_config(
    real_tasks_count=49,
    golden_tasks_count=1,
)

In [ ]:
main_pool = toloka_client.create_pool(main_pool)

Часть настроек основного пула была выполнена через UI.

In [ ]:
golden_tasks = [
    toloka.task.Task(
        pool_id=main_pool.id,
        input_values={
        INPUT_URL_COL: row[INPUT_URL_COL]
        },
        known_solutions = [
            toloka.task.BaseTask.KnownSolution(
                output_values={RESULT_COL: row[LABEL_COL]}
            )
        ],
        infinite_overlap=True
    )
    for _, row in honeypots.iterrows()
]

tasks = [
    toloka.task.Task(
        pool_id=main_pool.id,
        input_values={
        INPUT_URL_COL : row[INPUT_URL_COL]
        },
    )
    for _, row in main_data.iterrows()
]

main_tasks = toloka_client.create_tasks(golden_tasks + tasks, allow_defaults=True)
print(len(main_tasks.items))

100%|██████████| 100/100 [14:50<00:00,  8.91s/it]


9378


Далее идут старые попытки сделать много непересекающихся пулов. От этой идеи мы отошли, когда придумали оптимизировать траты благодаря эмбедингам.

In [ ]:
path_to_unlabeled_dataset = ""  # первой строкой там идет хэдер


data_to_label_chunks = list(pd.read_csv(path_to_unlabeled_dataset, chunksize=5000))

data_to_label_chunks = data_to_label_chunks[:-2] + [pd.concat(data_to_label_chunks[-2:])]

for chunk in data_to_label_chunks:
    print(chunk.shape)

*main_chunks, debug_chunks = data_to_label_chunks

(5000, 1)
(5000, 1)
(5000, 1)
(5000, 1)
(5000, 1)
(8044, 1)


In [ ]:
def create_skill_for_entry(pool_idx: int) -> toloka.Skill:
    skill_name = f"pool_{pool_idx}_participant"
    pool_i_participant_skill = next(toloka_client.get_skills(skill_name), None)

    if pool_i_participant_skill is None:
        pool_i_participant_skill = toloka_client.create_skill(
            name=skill_name,
            hidden=True,
    )
    skill_name
    return pool_i_participant_skill



In [ ]:
separate_pools_entries_skills = [
    create_skill_for_entry(i) for i, _ in enumerate(data_to_label_chunks)
]

separate_pools_entries_skills[:5]

[Skill(_unexpected={'deprecated': False, 'global': False}, name='pool_0_participant', private_comment=None, hidden=True, skill_ttl_hours=None, training=None, public_name={'EN': 'pool_0_participant'}, public_requester_description=None, owner=Owner(_unexpected={}, id='391c0230d63453b52e888f2324f4a8de', myself=True, company_id='151'), id='10566', created=datetime.datetime(2024, 12, 8, 17, 45, 39, 493000, tzinfo=datetime.timezone.utc)),
 Skill(_unexpected={'deprecated': False, 'global': False}, name='pool_1_participant', private_comment=None, hidden=True, skill_ttl_hours=None, training=None, public_name={'EN': 'pool_1_participant'}, public_requester_description=None, owner=Owner(_unexpected={}, id='391c0230d63453b52e888f2324f4a8de', myself=True, company_id='151'), id='10567', created=datetime.datetime(2024, 12, 8, 17, 45, 40, 28000, tzinfo=datetime.timezone.utc)),
 Skill(_unexpected={'deprecated': False, 'global': False}, name='pool_2_participant', private_comment=None, hidden=True, skill_

In [ ]:
# from functools import reduce

# def get_pool_i_filter(pools_participants_skills: toloka.Skill):
#     return reduce(lambda x, y: x & (toloka.filter.Skill(y.id) != None), # не был в пуле i \forall i
#                   pools_participants_skills,
#                   ((toloka.filter.Languages.in_('RU')) & (toloka.filter.Skill(EXAM_SKILL.id) == 1)),
#                   )


# # также понадобится контролль качества - быстрые ответы, ханипоты и конечно же выставление навыка для принадлежности к пулу

# def add_action_for_pool_i(pool_i: toloka.Pool, pool_idx: int) -> None:
#     pool_i.quality_control.add_action(
#         collector=toloka.collectors.AssignmentSubmitTime(history_size=5, fast_submit_threshold_seconds=30),
#         conditions=[
#         toloka.conditions.TotalSubmittedCount >= 3,
#         toloka.conditions.FastSubmittedCount >= 2,
#         ],
#         action=toloka.actions.RestrictionV2(
#             scope='POOL',
#             duration=3,
#             duration_unit='HOURS',
#             private_comment='Fast answers'
#         )
#     )

#     pool_i.quality_control.add_action(
#         collector=toloka.collectors.GoldenSet(history_size=5),
#         conditions=[
#             toloka.conditions.GoldenSetCorrectAnswersRate > 75,
#             toloka.conditions.GoldenSetAnswersCount >= 5
#         ],
#         action=toloka.actions.RestrictionV2(
#             scope='POOL',
#             duration=3,
#             duration_unit='HOURS',
#             private_comment='Bad control on honeypots'
#         )
#     )

#     pool_i.quality_control.add_action(
#         collector=toloka.collectors.AnswerCount(),
#         conditions=[
#             toloka.conditions.AssignmentsAcceptedCount > 0,
#         ],
#         action=toloka.actions.SetSkill(skill_id=separate_pools_entries_skills[pool_idx],
#                                        skill_value=1)
#     )

#     print(f"Quality control for pool {pool_idx} is configured")


In [ ]:
# def create_pool(pool_idx: int, honeypots: pd.DataFrame,
#                 ) -> tp.Generator[toloka.Pool, toloka.Pool, toloka.Pool]:
#     pool_i = toloka.Pool(
#         project_id=new_project.id,
#         private_name=f'MAIN pool {pool_idx}/{len(separate_pools_entries_skills) - 1}' + datetime.datetime.now().strftime('%Y-%m-%d'),
#         may_contain_adult_content=True,
#         reward_per_assignment=1,
#         assignment_max_duration_seconds=60*10,
#         will_expire=datetime.datetime.utcnow() + datetime.timedelta(days=365),
#         defaults=toloka.pool.Pool.Defaults(default_overlap_for_new_task_suites=3),
#     )
#     pool_i.set_mixer_config(
#         real_tasks_count=50, # TODO specify needed amount!!!!!
#         golden_tasks_count=5,
#     )

#     pool_i.filter = get_pool_i_filter(separate_pools_entries_skills)
#     add_action_for_pool_i(pool_i=pool_i, pool_idx=pool_idx)

#     pool_i = yield pool_i
#     print(pool_i)

#     pool_assignment_chunk: pd.DataFrame = data_to_label_chunks[pool_idx]

#     real_tasks = [
#         toloka.Task(
#             pool_id=pool_i.id,
#             input_values={INPUT_URL_COL: row[INPUT_URL_COL]}
#         )
#         for _, row in pool_assignment_chunk.iterrows()
#     ]
#     heneypots_tasks = [
#         toloka.Task(
#             pool_id=pool_i.id,
#             input_values={INPUT_URL_COL: row[INPUT_URL_COL]},
#             known_solutions = [
#                 toloka.task.BaseTask.KnownSolution(
#                     output_values={RESULT_COL: row[LABEL_COL]}
#                 )
#             ]
#         )
#         for _, row in honeypots.iterrows()
#     ]


#     toloka_client.create_tasks(real_tasks + heneypots_tasks, allow_defaults=True)
#     print(f"Tasks for pool {pool_idx} were uploaded")
#     return pool_i


Для примера я создал дебаг-пул (последний индекс)

In [ ]:
# pool_creator = create_pool(pool_idx=0, honeypots=exam_data)
# pool_to_be_created = next(pool_creator)
# created_pool = toloka_client.create_pool(pool_to_be_created)
# created_pool = pool_creator.send(created_pool)

Quality control for pool 0 is configured
FilterAnd(_unexpected={}, and_=[Languages(_unexpected={}, operator=<InclusionOperator.IN: 'IN'>, value='RU'), Skill(_unexpected={}, key='10562', operator=<CompareOperator.EQ: 'EQ'>, value=1), Skill(_unexpected={}, key='10566', operator=<CompareOperator.NE: 'NE'>, value=None), Skill(_unexpected={}, key='10567', operator=<CompareOperator.NE: 'NE'>, value=None), Skill(_unexpected={}, key='10568', operator=<CompareOperator.NE: 'NE'>, value=None), Skill(_unexpected={}, key='10569', operator=<CompareOperator.NE: 'NE'>, value=None), Skill(_unexpected={}, key='10570', operator=<CompareOperator.NE: 'NE'>, value=None), Skill(_unexpected={}, key='10571', operator=<CompareOperator.NE: 'NE'>, value=None), Skill(_unexpected={}, key='10572', operator=<CompareOperator.NE: 'NE'>, value=None), Skill(_unexpected={}, key='10573', operator=<CompareOperator.NE: 'NE'>, value=None), Skill(_unexpected={}, key='10574', operator=<CompareOperator.NE: 'NE'>, value=None), Sk

InternalApiError: You have got a(n) InternalApiError with http status code: 500
Code of error: INTERNAL_ERROR
Error details: Internal Error
request id: 1733682763569024-1638728351177782266. It needs to be specified when contacting support.

Ловлю ошибки, в начале думал, что это из-за того, что внутри функции все делаю, но непонятно

In [ ]:
# pool_idx = -1

# pool_i = toloka.Pool(
#     project_id=new_project.id,
#     private_name=f'MAIN pool {pool_idx}/{len(separate_pools_entries_skills) - 1}   от ' + datetime.datetime.now().strftime('%Y-%m-%d'),
#     may_contain_adult_content=True,
#     reward_per_assignment=1,
#     assignment_max_duration_seconds=60*10,
#     will_expire=datetime.datetime.utcnow() + datetime.timedelta(days=365),
#     defaults=toloka.pool.Pool.Defaults(default_overlap_for_new_task_suites=3),
# )
# pool_i.set_mixer_config(
#     real_tasks_count=50, # TODO specify needed amount!!!!!
#     golden_tasks_count=5,
# )

# pool_i.filter = reduce(
#         lambda x, y: x & (toloka.filter.Skill(y.id) != None), # не был в пуле i \forall i
#         separate_pools_entries_skills,
#         ((toloka.filter.Languages.in_('RU')) & (toloka.filter.Skill(EXAM_SKILL.id) == 1)),
#         )

# pool_i.quality_control.add_action(
#     collector=toloka.collectors.AssignmentSubmitTime(history_size=5, fast_submit_threshold_seconds=30),
#     conditions=[
#     toloka.conditions.TotalSubmittedCount >= 3,
#     toloka.conditions.FastSubmittedCount >= 2,
#     ],
#     action=toloka.actions.RestrictionV2(
#         scope='POOL',
#         duration=3,
#         duration_unit='HOURS',
#         private_comment='Fast answers'
#     )
# )

# pool_i.quality_control.add_action(
#     collector=toloka.collectors.GoldenSet(history_size=5),
#     conditions=[
#         toloka.conditions.GoldenSetCorrectAnswersRate > 75,
#         toloka.conditions.GoldenSetAnswersCount >= 5
#     ],
#     action=toloka.actions.RestrictionV2(
#         scope='POOL',
#         duration=3,
#         duration_unit='HOURS',
#         private_comment='Bad control on honeypots'
#     )
# )

# pool_i.quality_control.add_action(
#     collector=toloka.collectors.AnswerCount(),
#     conditions=[
#         toloka.conditions.AssignmentsAcceptedCount > 0,
#     ],
#     action=toloka.actions.SetSkill(skill_id=separate_pools_entries_skills[pool_idx],
#                                     skill_value=1)
# )

# print(f"Quality control for pool {pool_idx} is configured")



# pool_i = toloka_client.create_pool(pool_i)
# print(pool_i)

# pool_assignment_chunk: pd.DataFrame = data_to_label_chunks[pool_idx]

# real_tasks = [
#     toloka.Task(
#         pool_id=pool_i.id,
#         input_values={INPUT_URL_COL: row[INPUT_URL_COL]}
#     )
#     for _, row in pool_assignment_chunk.iterrows()
# ]
# heneypots_tasks = [
#     toloka.Task(
#         pool_id=pool_i.id,
#         input_values={INPUT_URL_COL: row[INPUT_URL_COL]},
#         known_solutions = [
#             toloka.task.BaseTask.KnownSolution(
#                 output_values={RESULT_COL: row[LABEL_COL]}
#             )
#         ]
#     )
#     for _, row in exam_data.iterrows()
# ]


# toloka_client.create_tasks(real_tasks + heneypots_tasks, allow_defaults=True)
# print(f"Tasks for pool {pool_idx} were uploaded")


Quality control for pool -1 is configured


InternalApiError: You have got a(n) InternalApiError with http status code: 500
Code of error: INTERNAL_ERROR
Error details: Internal Error
request id: 1733682991093533-14668145782227732197. It needs to be specified when contacting support.

## Получение результата из одного пула и получение финальной аггрегации

In [ ]:
WORKER_COL = "worker"
TASK_COL = "task"


def get_assignments(pool_id: str):
    answers_df = toloka_client.get_assignments_df(pool_id)
    answers_df = answers_df.rename(columns={
        f'INPUT:{INPUT_URL_COL}': TASK_COL,
        f'OUTPUT:{RESULT_COL}': LABEL_COL,
        'ASSIGNMENT:worker_id': WORKER_COL,
    })[[TASK_COL, LABEL_COL, WORKER_COL]]

    return answers_df


def aggregate_answers(answers_df: pd.DataFrame, aggregation_method: tp.Literal["majority-vote", "dawid-skene"]):
    aggregator = MajorityVote() if aggregation_method == "majority-vote" else DawidSkene()
    results = aggregator.fit_predict(data=answers_df)
    results = pd.DataFrame({INPUT_URL_COL: results.index, RESULT_COL: results.values})
    return results




In [ ]:
pool_id = ...
answers_for_pool = get_assignments(pool_id)
aggregated_answers = aggregate_answers(answers_for_pool, "dawid-skene")